# data_collection.py
Αρχικά κάνουμε import τα ακόλουθα:

- requests: Για να πραγματοποιήσουμε GET requests στο [Wikipedia](https://en.wikipedia.org/wiki/) και να πάρουμε την html την οποία θα επεξεργαστούμε/κάνουμε parse
- BeautifulSoup: Από το module bs4 για να κάνουμε parse την html που λάβαμε από το GET request και να κάνουμε extract το κείμενο από την html
- json: Ώστε να μετατρέψουμε δεδομένα όπως ο τίτλος κ το κείμενο σε μορφή json η οποία θα χρησιμοποιηθεί από το `indexing.py` για την δημιουργία ενός ανεστραμμένου ευρετηρίου

In [2]:
import requests
from bs4 import BeautifulSoup
import json

## collect_data
Αυτό το section περιγράφει την συνάρτηση `collect_data()`. Η συνάρτηση αυτή αποτελεί την κύρια και μοναδική λειτουργία του αρχείου `data_collection.py`.
Η συνάρτηση αρχικά δημιουργεί μια λίστα η οποία έχει όλα τα topics τα οποία συλλέγει ο 'crwaler'.

**Σημείωση**: Έχουμε επιλέξει συγκεκριμένα topics αντί να κάνουμε crawl όλη την πλατφόρμα του Wikipedia ώστε να χρειάζεται λιγότερος χρόνος κατα την συλλογή, την επεξεργασία αλλά κ την αναζήτηση. Συνεπώς υλοποιήσαμε ένα συνδυασμό των επιλογών που μας δόθηκαν στην εκφώνηση.

Στην συνέχεια θέτουμε το `base_url` ως: `https://en.wikipedia.org/wiki/` το οποίο υλοποιεί ένα είδος GET API μέσα από το οποίο θα πραγματοποιηθεί η συλλογή της html και στην συνέχεια του κειμένου. Επίσης, αρχικοποιούμε μια λίστα `documents` της οποίας κάθε στοιχείο είναι ένα dictionary(το υλοποιήσαμε κατα αυτό τον τρόπο για ευκολότερη προσπέλαση αργότερα) που διαθέτει το URL και το κείμενο του άρθρου.

Μετά χρησιμοποιούμε ενα for loop με το οποίο κάνουμε 'enumerate' τα topics.

Για κάθε topic φτιάχνουμε κατάλληλα το URL όπως περιγράφηκε παραπάνω, κάνουμε ένα GET request όπως περιγράφηκε παραπάνω για να πάρουμε την HTML του άρθρου και με την χρήση του `BeautifulSoup` κάνουμε χρήση του βασικού HTML parser και 'επιλέγουμε' όλα τα paragraphs (elements που χρησιμοποιούν το `<p>` tag), ενώνουμε όλα τα paragraphs με χρήση ενός new line για κάθε paragraph.

Τέλος χρησιμοποιούμε την μέθοδο `json.dumps` ώστε να γράψουμε σε ένα αρχείο raw json. Το αρχείο αυτό θα χρησιμοποιηθεί ως input 

In [3]:
def collect_data():
    # Define the topics to crawl related to Metal Music
    topics = [
        "Heavy_metal_music",
        "Powerwolf",
        "Thrash_metal",
        "Death_metal",
        "Black_metal",
        "Power_metal",
        "Metallica",
        "Iron_Maiden",
        "Black_Sabbath",
        "Slayer",
        "Megadeth",
        "Origins_of_heavy_metal",
        "New_Wave_of_British_Heavy_Metal",
        "Master_of_Puppets",
        "The_Number_of_the_Beast",
        "Paranoid_(album)",
        "Ozzy_Osbourne",
        "James_Hetfield",
        "Bruce_Dickinson",
        "Lemmy"
    ]
    
    base_url = "https://en.wikipedia.org/wiki/"
    documents = []
    
    for topic in topics:
        url = base_url + topic
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract text
        paragraphs = soup.find_all('p')
        document = "\n".join([para.text for para in paragraphs])
        documents.append({"url": url, "content": document})
    
    # Save to JSON
    with open('data.json', 'w') as f:
        json.dump(documents, f)

Για σκοπούς debugging και για να μπορούμε να τρέχουμε τον collector ως individual application προσθέσαμε το ακόλουθο:

In [ ]:
if __name__ == "__main__":
    collect_data()

# text_preprocessing.py

Κάνουμε import τα ακόλουθα:

- json: Για να γράψουμε νέο json αρχείο το οποίο περιλαμβάνει τα δεδομένα από τα άρθρα
- word_tokenize: Μέθοδος από το nltk για να σπάσουμε τα άρθρα σε individual tokens
- stopwords: Χρησιμοποιούμε το stopwords για να κάνουμε parse και τα stopwords
- PorterStemmer: Για να κάνουμε stem τις λέξεις και να αναγάγουμε τις κλιτές λέξεις

In [ ]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## preprocess_text

Με την ακόλουθη συνάρτηση διαβάζουμε το αρχείο `data.json` (αρχείο που περιλαμβάνει όλα τα άρθρα και τα αντίστοιχα URLs) και δημιουργούμε ένα set που έχει όλα τα `stopwords` του nltk (ενδιάμεσες μη-χαρακτηριστικές λέξεις στα Αγγλικά).

Για κάθε μέρος του json file ελέγχουμε αν είμαστε στο content ή στο URL.
Αν είμαστε στο content τότε χρησιμοποιούμε το `word_tokenize` για να πάρουμε τα individual tokens από το άρθρο, δημιουργούμε μια λίστα (`filtered_tokens`) η οποία περιέχει όλες τις λέξεις που δεν είναι stopwords δηλαδή είναι λέξεις που "περιέχουν" πληροφορία (αφαιρούμε τα stopwords). Στην συνέχεια κάνουμε stem κάθε λέξη από την παραπάνω λίστα για να βρούμε την ρίζα της κάθε λέξης κ να μπορέσουμε να την επεξεργαστούμε καλύτερα.
Τέλος αποθηκεύουμε τα "νεα" tokens στο αρχείο `preprocessed_data.json` το οποίο περιέχει τα περιεχόμενα του `data.json` μαζί με τα tokens σε αντίστοιχο json element.


In [ ]:
def preprocess_text():
    with open('data.json', 'r') as f:
        data = json.load(f)
    
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    for document in data:
        text = document['content']
        
        # Tokenization
        tokens = word_tokenize(text)
        
        # Stop-word removal
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        
        # Stemming
        stemmed_tokens = [ps.stem(word) for word in filtered_tokens]
        
        document['tokens'] = stemmed_tokens
    
    # Save preprocessed data
    with open('preprocessed_data.json', 'w') as f:
        json.dump(data, f)

# indexing.py

Κάνουμε import τα ακόλουθα:

-  json: Για να επεξεργαζόμαστε json files
-  defaultdict: Για να δημιουργήσουμε το ανάστροφο ευρετήριο που περιέχει λίστες ως default values για κάθε key (κάθε νεο token έχει μια λίστα που τα στοιχεία της είναι οι θέσεις στην οποία βρίσκεται το token).

In [ ]:
import json
from collections import defaultdict

## create_index

Στην συνάρτηση αυτή χρησιμοποιούμε το αρχείο `preprocessed_data.json` που δημιουργήσαμε παραπάνω. Διατρέχουμε το αρχείο και κρατάμε το `tokens` element από το json. Για κάθε token κάνουμε την διαδικασία ένταξης του στο ανάστροφο ευρετήριο.
Τέλος, αποθηκεύουμε το ανάστροφο ευρετήριο στο αρχείο `index.json`.

In [ ]:
def create_index():
    with open('preprocessed_data.json', 'r') as f:
        data = json.load(f)
    
    inverted_index = defaultdict(list)
    
    for doc_id, document in enumerate(data):
        for token in document['tokens']:
            inverted_index[token].append(doc_id)
    
    # Save index
    with open('index.json', 'w') as f:
        json.dump(inverted_index, f)

# query_processing

Το module `query_processing` περιλαμβάνει πολλαπλές συναρτήσεις για την εκτέλεση βημάτων υποδομής για την σωστή έναρξη της μηχανής, οι οποίες είναι:
- process_query: Κάνουμε κανονικοποίηση του query από τον χρήστη 
- get_documents: Διαβάζουμε όλα τα δεδομένα του `data.json` και τα αποθηκεύουμε σε μια λίστα που περιέχει dictionaries (τρόπος να έχουμε JSON στην Python)
- search: Κάνουμε process το query του χρήστη, ελέγχουμε την μέθοδο ανάκτησης (ποια είναι και αν είναι valid το user input) και διαβάζουμε όλα τα δεδομένα του `data.json`

## process_query:

Αρχικά λαμβάνουμε ως parameter το query προς αναζήτηση από τον χρήστη (την συνάρτηση `user_interface`), το input αυτό το κάνουμε tokenize (αφαιρούμε τα κενά και σπάμε σε tokens). Στην συνέχεια, αφαιρούμε τα stopwords ώστε να κρατήσουμε μόνο τις χαρακτηριστικές λέξεις για να έχουμε καλύτερα αποτελέσματα. Τέλος κάνουμε stemming τις χαρακτηριστικές λέξεις ώστε να μπορούμε να επιστρέψουμε στον χρήστη πιθανά άρθρα που έχουν το query χωρίς κλήση.

In [ ]:
def process_query(query):
    # Preprocess query
    tokens = word_tokenize(query)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in filtered_tokens]
    
    return stemmed_tokens

## get_documents:

Δημιουργούμε μια λίστα από dictionaries που περιέχουν όλα τα κείμενα (χωρις URLs ή Links) του `data.json`.

In [ ]:
def get_documents(doc_ids):
    with open('data.json', 'r') as f:
        data = json.load(f)
    return [data[doc_id]['content'] for doc_id in doc_ids]

## search:

Η search χρησιμοποιείται ώστε να πετύχουμε τα ακόλουθα:
1. Κλήση `process_query`, για να κάνουμε normalize το user query
2. Αναγνώριση του αλγορίθμου που ζήτησε ο χρήστης
3. Ανάγνωση και αποστολή των κειμένων μαζί με τα DocIDs στο ανάστροφο ευρετήριο

In [ ]:
def search(query, method="boolean"):
    query_tokens = process_query(query)
    
    if method == "boolean":
        from ranking import boolean_retrieval
        doc_ids = boolean_retrieval(query_tokens)
    elif method == "vsm":
        from ranking import vector_space_model
        doc_ids = vector_space_model(query_tokens)
    elif method == "bm25":
        from ranking import bm25
        doc_ids = bm25(query_tokens)
    else:
        raise ValueError("Unknown method: {}".format(method))
    
    documents = get_documents(doc_ids)
    return documents

# user_interface

Η συνάρτηση `user_interface` χρησιμοποιείται ως μια απλή tty διεπαφή με τον χρήσητ όπου ο χρήστης καλείται να δώσει ένα κείμενο προς αναζήτηση και στην συνέχεια καλείται να διαλέξει τον αλγόριθμο αναζήτησης.
Σε περίπτωση όπου είναι λάθος η είσοδος για τον αλγόριθμο αναζήτησης τότε επιλέγουμε αυθαίρετα τον αλγόριθμο `Boolean Retrieval`.

In [ ]:
from query_processing import search

def user_interface():
    query = input("Enter your search query: ")
    print("Select retrieval method:")
    print("1. Boolean Retrieval")
    print("2. Vector Space Model (VSM)")
    print("3. Probabilistic Retrieval (BM25)")
    method_choice = input("Enter the number corresponding to your choice: ")

    if method_choice == "1":
        method = "boolean"
    elif method_choice == "2":
        method = "vsm"
    elif method_choice == "3":
        method = "bm25"
    else:
        print("Invalid choice. Defaulting to Boolean Retrieval.")
        method = "boolean"

    results = search(query, method)
    print(f"Search Results for '{query}' using {method} method:")
    for result in results:
        print(result)

# main.py

Κάνουμε import τα ακόλουθα:

- data_collection: Για να συλλέξουμε δεδομένα από το wikipedia και να αποθηκεύσουμε το raw json σε κατάλληλο αρχείο `data.json`
- text_preprocessing:
- indexing:
- user_interface:

In [ ]:
from data_collection import collect_data
from text_preprocessing import preprocess_text
from indexing import create_index
from user_interface import user_interface

Δημιουργούμε την συνάρτηση main και στην συνέχεια θα την καλέσουμε.
Η main χρησιμοποιείται για την ενορχήστρωση των άλλων αρχείων και πραγματοποιείται η κλήση συναρτήσεων όπως περιγράφηκε στο παραπάνω section

In [ ]:
def main():
    collect_data()
    preprocess_text()
    create_index()
    user_interface()

# Tests

1.  Χρήση λέξης 'lemmy'.

Γνωρίζουμε οτι η λέξη υπάρχει, είναι χαρακτηριστική ενός ή παραπάνω topics και το keyword αναζητήθηκε είναι case in-sensitive

**Αναμενόμενα αποτελέσματα**: Όλα τα άρθρα που περιλαμβάνουν την λέξη 'lemmy' μέσα στο άρθρο

In [ ]:
main()

Select retrieval method:
1. Boolean Retrieval
2. Vector Space Model (VSM)
3. Probabilistic Retrieval (BM25)
Search Results for 'lemmy' using boolean method:


Heavy metal (or simply metal) is a genre of rock music that developed in the late 1960s and early 1970s, largely in the United Kingdom and United States.[2] With roots in blues rock, psychedelic rock and acid rock, heavy metal bands developed a thick, monumental sound characterized by distorted guitars, extended guitar solos, emphatic beats and loudness.

In 1968, three of the genre's most famous pioneers – British bands Led Zeppelin, Black Sabbath and Deep Purple – were founded.[3] Though they came to attract wide audiences, they were often derided by critics. Several American bands modified heavy metal into more accessible forms during the 1970s: the raw, sleazy sound and shock rock of Alice Cooper and Kiss; the blues-rooted rock of Aerosmith; and the flashy guitar leads and party rock of Van Halen.[4] During the mid-1970s, Jud

2. Χρήση λέξης 'a'

Θεωρούμε οτι η λέξη 'a' υπάρχει στα άρθρα αλλά δεν είναι χαρακτηριστική.

**Αναμενόμενα αποτελέσματα**: Θα έχουμε error καθώς οι μη-χαρακτηριστικές λέξεις έχουν αφαιρεθεί, συνεπώς στο ανάστροφο ευρετήριο δεν θα βρεθεί η λέξη.

In [ ]:
main() # Δεν κρατήσαμε αποτελέσματα καθώς γίνονται disclosed paths και stack traces

3. Χρήση λέξης '2012'.

Γνωρίζουμε οτι η λέξη '2012' ανήκει σε ένα ή περισσότερα άρθρα και μπορεί να χρησιμοποιηθεί ως χαρακτηριστική καθώς είναι περιορισμένη η χρήση της.

**Αναμενόμενα αποτελέσματα**: Θα επιστραφούν όλα τα άρθρα στα οποία υπάρχει η λέξη/έτος '2012'.

In [ ]:
main()

Select retrieval method:
1. Boolean Retrieval
2. Vector Space Model (VSM)
3. Probabilistic Retrieval (BM25)
Search Results for '2012' using boolean method:


Powerwolf (often stylized in all caps) is a German power metal band founded in 2003 in Saarbrücken by members of Red Aim. The band consists of vocalist Karsten Brill as "Attila Dorn", lead guitarist Benjamin Buss as "Matthew Greywolf", bassist/rhythm guitarist David Vogt as "Charles Greywolf", keyboardist Christian Jost as "Falk Maria Schlegel" and drummer Roel van Helden. The group uses religious themes and images, contrary to traditional power metal music, as well as corpse paint, gothic-tinged compositions and songs about werewolf and vampire legends.

They released their debut album, Return in Bloodred, in 2005. Their second album, Lupus Dei, was released in 2007. The band entered the official German charts for the first time in 2009 with their third album, Bible of the Beast. Shortly after its release, their first drummer Ste

4. Χρήση της λέξης 'Power wolf'.

Γνωρίζουμε οτι η λέξη 'Power wolf' γίνεται χαρακτηριστική μετά την κανονικοποίηση της εισόδου. Γνωρίζουμε οτι και το input αλλά και το κείμενο γίνεται κανονικοποιημένο.

**Αναμενόμενα αποτελέσματα**: Θεωρώντας τα παραπάνω σχόλια περιμένουμε να μας επιστραφούν όλα τα άρθρα που περιλαμβάνουν την λέξη 'powerwolf' (Η λέξη θα γίνει χωρίς spaces/κενά και θα είναι lowercase λόγω κανονικοποίησης)

In [ ]:
main()

Select retrieval method:
1. Boolean Retrieval
2. Vector Space Model (VSM)
3. Probabilistic Retrieval (BM25)
Search Results for 'Power wolf' using boolean method:


Black Sabbath were an English rock band formed in Birmingham in 1968 by guitarist Tony Iommi, drummer Bill Ward, bassist Geezer Butler, and vocalist Ozzy Osbourne. They are often cited as pioneers of heavy metal music.[1] The band helped define the genre with their first three albums Black Sabbath, Paranoid (both 1970), and Master of Reality (1971). Following Osbourne's departure in 1979, the band underwent multiple line-up changes, with Iommi being the only constant member throughout its history.

After previous iterations of the group – the Polka Tulk Blues Band, and Earth – the band settled on the name Black Sabbath in 1969. They distinguished themselves through occult themes with horror-inspired lyrics and down-tuned guitars. Signing to Philips Records in November 1969, they released their first single, a cover of the Cr

5. Χρήση της λέξης '1945 -'

Η λέξη '1945 -' περιλαμβάνει punctuation (τον χαρακτήρα ' -') και γνωρίζουμε οτι ανήκει σε ένα άρθρο, λόγω κανονικοποίησης 

**Αναμενόμενα αποτελέσματα**: Γνωρίζουμε οτι τα tokens '1945 -' ανήκει σε ένα ή περισσότερα άρθρα συνεπώς περιμένουμε να εμφανιστούν όλα τα άρθρα που περιλαμβάνουν τα tokens '1945 -' και '1945-'. 

In [ ]:
main()

Select retrieval method:
1. Boolean Retrieval
2. Vector Space Model (VSM)
3. Probabilistic Retrieval (BM25)
Search Results for '1945 -' using boolean method:


Ian Fraser Kilmister (24 December 1945 – 28 December 2015), better known as Lemmy Kilmister or simply Lemmy, was a British musician. He was the founder, lead singer, bassist and primary songwriter of the rock band Motörhead, of which he was the only continuous member, and a member of Hawkwind from 1971 to 1975.

A foundational force in the genre following the advent of the new wave of British heavy metal, Lemmy was known for his appearance, which included his signature friendly mutton chops, his military-influenced fashion sense and his gravelly rasp of a voice. It was once declared "one of the most recognisable voices in rock".[1] He was also noted for his unique posture when singing, which was once described as "looking up towards a towering microphone tilted down into his weather-beaten face".[2] He was also known for his bas